<a href="https://colab.research.google.com/github/merucode/RL/blob/32-Study-Udemy-Policy_gradient/3_advantage_actor_critic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advantage Actor-Critic


In [ ]:
!apt-get install -y xvfb

!pip install \
  gymnasium==0.26.3 \
  pygame \
  pytorch-lightning==1.6 \
  pyvirtualdisplay

#### Setup virtual display

In [ ]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

#### Import the necessary code libraries

In [ ]:
import copy
import torch
import random
import gym
import matplotlib

import numpy as np
import matplotlib.pyplot as plt

import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from torch.distributions import Normal

from pytorch_lightning import LightningModule, Trainer

from gym.wrappers import RecordVideo, RecordEpisodeStatistics, NormalizeObservation


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

In [ ]:
@torch.no_grad()
def test_env(env_name, policy, obs_rms, **kwargs):
  env = gym.make(env_name, **kwargs)
  env = RecordVideo(env, 'videos', episode_trigger=lambda e: True)
  env = NormalizeObservation(env)
  env.obs_rms = obs_rms
  policy = policy.to(device)

  for episode in range(10):
    done = False
    obs = env.reset()
    while not done:
      loc, scale = policy(obs)
      action = torch.normal(loc, scale)
      action = action.cpu().numpy()
      obs, _, done, _ = env.step(action)
  del env


def display_video(episode=0):
  video_file = open(f'/content/videos/rl-video-episode-{episode}.mp4', "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"<video width=600 controls><source src='{video_url}'></video>")

#### Create the policy

In [ ]:
class GradientPolicy(nn.Module):

  def __init__(self, in_features, out_dims, hidden_size=128):
    super().__init__()
    self.fc1 = nn.Linear(in_features, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc_mu = nn.Linear(hidden_size, out_dims)
    self.fc_std = nn.Linear(hidden_size, out_dims)

  def forward(self, x):
    x = torch.tensor(x).float().to(device)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    loc = self.fc_mu(x)
    loc = torch.tanh(loc) * 2
    scale = self.fc_std(x)
    scale = F.softplus(scale) + 0.001
    return loc, scale

In [ ]:
class ValueNet(nn.Module):

  def __init__(self, in_features, hidden_size=128):
    super().__init__()
    self.fc1 = nn.Linear(in_features, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, 1)

  def forward(self, x):
    x = torch.tensor(x).float().to(device)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

#### Create the environment

In [ ]:
env = gym.make("Pendulum-v1")
env.reset()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(env.render(mode="rgb_array"))

In [ ]:
env = gym.vector.make("Pendulum-v1", num_envs=2)

In [ ]:
env.reset()

In [ ]:
env.observation_space, env.action_space

In [ ]:
next_obs, rewards, dones, infos = env.step(env.action_space.sample())

In [ ]:
infos

In [ ]:
def create_env(env_name, num_envs):
  env = gym.vector.make(env_name, num_envs)
  env = RecordEpisodeStatistics(env)
  env = NormalizeObservation(env)
  return env

#### Create the dataset

In [ ]:
class RLDataset(IterableDataset):

  def __init__(self, env, policy, steps_per_epoch):
    self.env = env
    self.policy = policy
    self.steps_per_epoch = steps_per_epoch
    self.obs = env.reset()

  @torch.no_grad()
  def __iter__(self):
    for step in range(self.steps_per_epoch):
      loc, scale = self.policy(self.obs)
      action = torch.normal(loc, scale)
      action = action.cpu().numpy()
      next_obs, reward, done, info = self.env.step(action)
      yield self.obs, action, reward, done, next_obs
      self.obs = next_obs

#### Create the A2C algorithm

In [ ]:
class A2C(LightningModule):

  def __init__(self, env_name, num_envs=64, samples_per_epoch=8,
               batch_size=1, hidden_size=64, policy_lr=1e-4, value_lr=1e-3,
               gamma=0.99, entropy_coef=0.01, optim=AdamW):

    super().__init__()

    self.env = create_env(env_name, num_envs=num_envs)

    obs_size = self.env.single_observation_space.shape[0]
    action_dims = self.env.single_action_space.shape[0]

    self.policy = GradientPolicy(obs_size, action_dims, hidden_size)
    self.value_net = ValueNet(obs_size, hidden_size)
    self.target_value_net = copy.deepcopy(self.value_net)

    self.dataset = RLDataset(self.env, self.policy, samples_per_epoch)

    self.save_hyperparameters()

  def configure_optimizers(self):
    value_opt = self.hparams.optim(self.value_net.parameters(), lr=self.hparams.value_lr)
    policy_opt = self.hparams.optim(self.policy.parameters(), lr=self.hparams.policy_lr)
    return value_opt, policy_opt

  def train_dataloader(self):
    return DataLoader(dataset=self.dataset, batch_size=self.hparams.batch_size)

  def training_step(self, batch, batch_idx, optimizer_idx):

    samples, envs = batch[0].shape[0:2]
    reshape_fn = lambda x: x.view(samples * envs, -1)
    obs_b, action_b, reward_b, done_b, next_obs_b = map(reshape_fn, batch)

    state_values = self.value_net(obs_b)

    with torch.no_grad():
      next_state_values = self.target_value_net(next_obs_b)
      next_state_values[done_b] = 0.0
      target = reward_b + self.hparams.gamma * next_state_values

    if optimizer_idx == 0:
      loss = F.smooth_l1_loss(state_values, target)
      self.log("episode/Value Loss", loss)
      return loss

    elif optimizer_idx == 1:
      advantages = (target - state_values).detach()

      loc, scale = self.policy(obs_b)
      dist = Normal(loc, scale)

      log_probs = dist.log_prob(action_b).sum(dim=-1, keepdim=True)
      entropy = dist.entropy().sum(dim=-1, keepdim=True)

      pg_loss = - log_probs * advantages
      loss = (pg_loss - self.hparams.entropy_coef * entropy).mean()

      self.log("episode/PG Loss", pg_loss.mean())
      self.log("episode/Entropy", entropy.mean())

      return loss

  def training_epoch_end(self, training_step_outputs):
    if self.current_epoch > 0 and self.current_epoch % 25 == 0:
      self.log("episode/Return", self.env.return_queue[-1])

    if self.current_epoch > 0 and self.current_epoch % 10 == 0:
      self.target_value_net.load_state_dict(self.value_net.state_dict())

#### Purge logs and run the visualization tool (Tensorboard)

In [ ]:
!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

#### Train the policy

In [ ]:
algo = A2C("Pendulum-v1")

trainer = Trainer(
    gpus=num_gpus,
    max_epochs=2000,
    log_every_n_steps=1
)

trainer.fit(algo)

#### Check the resulting policy

In [ ]:
import warnings
warnings.filterwarnings('ignore')

test_env('Pendulum-v1', algo.policy, algo.env.obs_rms)

In [ ]:
display_video(episode=5)